# Comments on "An Efficient Algorithm for Computing Free Distance"
## by Larsen

In [1]:
from enum import Enum
from copy import deepcopy
import networkx as nx

%run state_transition_diagram.ipynb

class state_type(Enum):
    DEAD_FORWARD = 0
    DEAD_BACKWARD = 1
    ALIVE_FORWARD = 2
    ALIVE_BACKWARD = 3

def hamming_weight(matrix):
    weight = 0
    for i in range(matrix.nrows()):
        for j in range(matrix.ncols()):
            if matrix[i, j] != 0:
                weight += 1
    return weight
    
def dec_to_bin(number, length):
    # computes the binary representation of a decimal number in the way we need it
    # for the registers
    str_list = list(bin(number))[2:]
    bin_list = [0 for _ in range(length-len(str_list))] + [int(s) for s in str_list]
    bin_list.reverse()
    return matrix(1, length, bin_list)

def coefficients_up_to_deg(f, n):
    # returns the coefficients of a polynomial up to the coefficient of degree n.
    field = f.parent().base()
    coeff_list = f.coefficients(sparse=False)
    coeff_list = coeff_list + [0 for _ in range(n - len(coeff_list))]
    return matrix(field, n, 1, coeff_list)

def degree(generator_matrix):
    # works only for row reduced matrices.
    row_degs = row_degrees(generator_matrix)
    degree = sum(row_degs)
    return degree

In [2]:
def main_loop_after_extension(n, q, zero_state, E, current_state, stored_states, diagram, W_star, final_round):
    forward_type = [state_type.DEAD_FORWARD, state_type.ALIVE_FORWARD]
    backward_type = [state_type.DEAD_BACKWARD, state_type.ALIVE_BACKWARD]
    dead_type = [state_type.DEAD_FORWARD, state_type.DEAD_BACKWARD]
    alive_type = [state_type.ALIVE_FORWARD, state_type.ALIVE_BACKWARD]
    stored_states_copy = deepcopy(stored_states)
    W_m = current_state[2]
    T_m = current_state[1]
    S_m = current_state[0]
    state_count_single_run = 0
    if current_state[1] == state_type.ALIVE_FORWARD:
        # We look for the correct edge and determine the next state including the weight 
        new_state = diagram.nodes[S_m]["next state"][E]
        next_state_E = [
            new_state, state_type.ALIVE_FORWARD,
            hamming_weight(diagram[S_m][new_state][0]["output"]) + W_m
        ]
    else:
        # We look for the correct edge and determine the next state including the weight
        # in this case we have to switch the arguments in diagram as we go from the previous state to S_m.
        new_state = diagram.nodes[S_m]["previous state"][E]
        next_state_E = [
            new_state, state_type.ALIVE_BACKWARD,
            hamming_weight(diagram[new_state][S_m][0]["output"]) + W_m
        ]
    S = next_state_E[0]
    W = next_state_E[2]
    if S == zero_state:
        if W < W_star:
            W_star = W
        return stored_states_copy, W_star, state_count_single_run
    goto15 = False
    # Step (6)
    if W > (W_star + n - 1) / 2:
        goto15 = True
    goto9 = False
    # Step (7)
    if not goto15:
        for state in stored_states_copy:
            if state[0] == S:
                S_k = state[0]
                T_k = state[1]
                W_k = state[2]
                goto9 = True
                break
    # Step (8)
    if not (goto9 or goto15):
        stored_states_copy.append([S, T_m, W])
        state_count_single_run += 1
        goto15 = True
    goto13 = False
    # Step (9)
    if not goto15:
        if (T_k in forward_type and T_m in forward_type) or (T_k in backward_type and T_m in backward_type):
            goto13 = True
    # Step (10) and (11)
    if not (goto13 or goto15):
        W_star = min(W_star, W + W_k)
        if W >= W_k:
            goto15 = True
    # Step (12)
    if not (goto13 or goto15):
        W_k = W
        if T_m in forward_type:
            if T_k in dead_type:
                T_k = state_type.DEAD_FORWARD
            else:
                T_k = state_type.ALIVE_FORWARD
        else:
            if T_k in dead_type:
                T_k = state_type.DEAD_BACKWARD
            else:
                T_k = state_type.ALIVE_BACKWARD          
        for state in stored_states_copy:
            if state[0] == S_k:
                state[2] = W_k
                state[1] = T_k
                break
    # Step (13)
    if not goto15:
        if T_k in dead_type:
            goto15 = True
    # Step (14)
    if not goto15:
        W_k = min(W_k, W)
        for state in stored_states_copy:
            if state[0] == S_k:
                state[2] = W_k
                break
    # Step (15) and (16)
    if final_round:
        if T_m in forward_type:
            T_m = state_type.DEAD_FORWARD
        else:
            T_m = state_type.DEAD_BACKWARD
        for state in stored_states_copy:
            if state[0] == S_m:
                state[1] = T_m
                break
    return stored_states_copy, W_star, state_count_single_run

In [3]:
def minimum_distance(generator_matrix, upper_bound="Singleton"):
    field = generator_matrix[0,0].parent().base()
    q = field.cardinality()
    polynomials = generator_matrix[0]
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    try:
        row_degs, states = get_states(generator_matrix)
    except:
        raise
    memory = max(row_degs)
    delta = degree(generator_matrix)
    if upper_bound == "Singleton":
        upper_bound = (n-k) * (delta//k + 1) + delta + 1
    # make generator polynomial matrix into the g_i
    g_i_list = [coefficients_up_to_deg(f, memory + 1) for f in polynomials]
    # generate the matrix to get the outputs for each node.
    G_M_n = block_matrix(1, n, g_i_list, subdivide=False)
    state_transition_diagram, backward_diagram = make_diagram(generator_matrix, states, row_degs)
    # here the main part of the algorithm starts.
    W_star = upper_bound
    # generate all possible extension for the all zero state (forwards and backwards)
    zero_state = tuple([tuple([0 for _ in range(row_degs[i])]) for i in range(k)])
    starting_state = zero_state
    next_states_forward = state_transition_diagram[starting_state]
    final_state = zero_state
    next_states_backward = backward_diagram[final_state]
    # stored states is the array in the algorithm.
    # every element in the array consists of the state, the type and the weight of the state.
    # we do not want to consider the extension from the zero state to the zero state,
    # that's why we have the if condition.
    stored_states = []
    for new_state, multiedge in next_states_forward.items():
        wt_output = hamming_weight(multiedge[0]["output"])
        if new_state != zero_state:
            # This is to take the row degree 0 case into account.
            state_already_stored = False
            for stored_state in stored_states:
                if new_state == stored_state[0]:
                    state_already_stored = True
                    if wt_output < stored_state[2]:
                        stored_state = [new_state, state_type.ALIVE_FORWARD, wt_output]
            if not state_already_stored:
                stored_states.append([new_state, state_type.ALIVE_FORWARD, wt_output])
        else:
            # From zero state to zero state with nonzero input.
            if wt_output != 0:
                W_star = min(W_star, wt_output)
    for new_state_backward, multiedge_backward in next_states_backward.items():
        if new_state_backward != zero_state:
            weight_backward = hamming_weight(multiedge_backward[0]["output"])
            state_already_stored = False
            for stored_state in stored_states:
                if new_state_backward == stored_state[0]:
                    weight_path = weight_backward + stored_state[2]
                    if weight_path < W_star:
                        W_star = weight_path
                    state_already_stored = True
                    if weight_backward < stored_state[2]:
                        stored_state = [new_state_backward, state_type.ALIVE_BACKWARD, weight_backward]
            if not state_already_stored:
                stored_states.append([new_state_backward, state_type.ALIVE_BACKWARD, weight_backward])     
    finished = False
    current_state = None
    state_count = 1
    # start with step (2)
    while not finished:
        W_m = upper_bound
        # num_non_dead is used to check condition (3) in the algorithm.
        num_non_dead = 0
        # We look for a non-dead state of minimal weight (Steps (2) and (3)).
        for state in stored_states:
            if state[1] == state_type.ALIVE_FORWARD or state[1] == state_type.ALIVE_BACKWARD:
                num_non_dead += 1
                if state[2] <= W_m:
                    W_m = state[2]
                    current_state = state
        W_m = current_state[2]
        T_m = current_state[1]
        S_m = current_state[0]
        # Step (3)
        if 2 * W_m >= W_star or num_non_dead <= 0:
            # Step (17)
            return W_star, state_count
        #make extensions.
        # at first we consider all possible inputs and then in the for loop we make the extension
        # according to the input and do the main part of the algorithm.
        possible_inputs = possibilities_inputs(generator_matrix, row_degs)
        for i in range(len(possible_inputs)):
            E = possible_inputs[i]
            # in the final round we have to make the current state dead.
            final_round = False
            if i == len(possible_inputs)-1:
                final_round = True
            stored_states, W_star, state_count_single_run = main_loop_after_extension(
                n, q, zero_state, E, current_state, stored_states,
                state_transition_diagram, W_star, final_round
            )
            state_count += state_count_single_run